In [38]:
from foreduce.vampire.parser import read_string

read_string("""
% Running in auto input_syntax mode. Trying TPTP
[SA] new: 1. lives(agatha) [input]
[SA] new: 2. lives(butler) [input]
[SA] new: 3. lives(charles) [input]
[SA] new: 4. ~richer(X0,X1) | ~killed(X0,X1) [input]
[SA] new: 5. ~hates(charles,X2) | ~hates(agatha,X2) [input]
[SA] new: 6. ~hates(X3,charles) | ~hates(X3,butler) | ~hates(X3,agatha) [input]
[SA] new: 7. hates(agatha,agatha) [input]
[SA] new: 8. hates(agatha,charles) [input]
[SA] new: 9. hates(X4,X5) | ~killed(X4,X5) [input]
[SA] new: 10. hates(butler,X6) | ~hates(agatha,X6) [input]
[SA] new: 11. hates(butler,X7) | richer(X7,agatha) | ~lives(X7) [input]
[SA] new: 12. goal_0 | killed(charles,agatha) | killed(butler,agatha) [input]
[SA] new: 13. ~goal_0 [input]

""")

(lives(agatha)
 lives(butler)
 lives(charles)
 ~richer(X0, X1) | ~killed(X0, X1)
 ~hates(charles, X2) | ~hates(agatha, X2)
 ~hates(X3, charles) | ~hates(X3, butler) | ~hates(X3, agatha)
 hates(agatha, agatha)
 hates(agatha, charles)
 hates(X4, X5) | ~killed(X4, X5)
 hates(butler, X6) | ~hates(agatha, X6)
 hates(butler, X7) | richer(X7, agatha) | ~lives(X7)
 goal_0 | killed(charles, agatha) | killed(butler, agatha)
 ~goal_0,
 [[], [], [], [], [], [], [], [], [], [], [], [], []])

In [1]:
VAMPIRE_PATH = '/home/apluska/.vampire/bin/vampire_z3_rel_static_casc2023_6749'
TPTP_PATH = '/home/apluska/TPTP-v8.2.0/'

In [2]:
import pexpect

In [17]:
process.close()

In [3]:
from foreduce.fol.logic import Problem

process = pexpect.spawn(
    VAMPIRE_PATH +\
    ' ./problems/PUZ/PUZ001-1.p' +\
    ' --manual_cs on' +\
    ' --show_new on' +\
    ' --proof off' +\
    ' --avatar off' +\
    ' -t 0'
)
i=1
problem, tree, mapping = Problem(), [], {}

In [4]:
from foreduce.vampire.parser import read_string

for _ in range(100):
    try:
        branch = process.expect('Pick a clause:\r\n')
        string = str(process.before, encoding='utf-8')
        string = string[string.find('\n')+1:].replace('\r\n', '\n')
        if string and not 'User error' in string:
            problem, tree, mapping = read_string(string, problem, tree, mapping)
        process.sendline(str(i))
        i += 1
    except pexpect.EOF:
        print('Done')
        break

Done


In [5]:
mapping

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18,
 20: 19,
 21: 20,
 22: 21,
 23: 22,
 24: 23,
 25: 24,
 26: 25,
 27: 26,
 28: 27,
 29: 28,
 30: 29,
 31: 30,
 32: 31}

In [6]:
tree

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [7, 5],
 [13, 6],
 [9, 6],
 [9, 7],
 [10, 5],
 [17, 2],
 [18, 15],
 [11, 8],
 [20, 12],
 [16, 5],
 [22, 15],
 [19, 10],
 [24, 1],
 [21, 4],
 [26, 6],
 [23, 10],
 [28, 1],
 [25, 3],
 [27, 8]]

In [179]:
string = str(process.before, encoding='utf-8')
string = string[string.find('\n')+1:].replace('\r\n', '\n')
string

'[SA] new: 33. ~killed(butler,agatha) [resolution 30,4]\n[SA] new: 34. $false [subsumption resolution 33,28]\n% Refutation found. Thanks to Tanya!\n% SZS status Unsatisfiable for PUZ001-1\n% ------------------------------\n% Version: Vampire 4.8 (commit 8d999c135 on 2023-07-12 16:43:10 +0000)\n% Linked with Z3 4.9.1.0 6ed071b44407cf6623b8d3c0dceb2a8fb7040cee z3-4.8.4-6427-g6ed071b44\n% Termination reason: Refutation\n\n% Memory used [KB]: 511\n% Time elapsed: 302.920 s\n% ------------------------------\n% ------------------------------\n31\n32\n33\n'

In [122]:
from foreduce.vampire.parser import read_string

problem, tree = read_string(string)

In [177]:
process.sendline(str(i))
i+=1

In [50]:
from foreduce.vampire.parser import read_string

read_string(

(lives(agatha)
 lives(butler)
 lives(charles)
 ~richer(X0, X1) | ~killed(X0, X1)
 ~hates(charles, X2) | ~hates(agatha, X2)
 ~hates(X3, charles) | ~hates(X3, butler) | ~hates(X3, agatha)
 hates(agatha, agatha)
 hates(agatha, charles)
 hates(X4, X5) | ~killed(X4, X5)
 hates(butler, X6) | ~hates(agatha, X6)
 hates(butler, X7) | richer(X7, agatha) | ~lives(X7)
 goal_0 | killed(charles, agatha) | killed(butler, agatha)
 ~goal_0,
 [[], [], [], [], [], [], [], [], [], [], [], [], []])

In [3]:
import subprocess
import os
from tqdm.auto import tqdm

outputs = []
for file in tqdm(os.listdir(TPTP_PATH + 'Problems/PUZ/')):
    args = [VAMPIRE_PATH, TPTP_PATH + 'Problems/PUZ/' + file,  '--show_new', 'on', '--include', TPTP_PATH, '-t', '0.1', '--avatar', 'off', '--proof', 'off']
    result = subprocess.run(args, capture_output=True, text=True, timeout=2)
    outputs.append(result.stdout)

/home/apluska/miniconda3/envs/foreduce/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/243 [00:00<?, ?it/s]

100%|██████████| 243/243 [00:23<00:00, 10.38it/s]


In [2]:
from foreduce.vampire.parser import read_file as read_vampire

problem, tree = read_vampire('./proofs/PUZ/PUZ001-1.p')

In [3]:
from foreduce.transformer.tokenizer import TokenConfig
from foreduce.data.data import ProofTokens

config = TokenConfig()
dataset = ProofTokens(config)
dataset.add_proof(problem, tree)

In [4]:
from foreduce.transformer.embedding import FormulaEmbedding

embedding = FormulaEmbedding(config)

/home/apluska/miniconda3/envs/foreduce/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
x, y, target = dataset[:2]

embedding(x).shape

torch.Size([2, 64])

In [2]:
from foreduce.transformer.tokenizer import ProofTokenizer, TokenConfig

ProofTokenizer(TokenConfig())(problem, tree)

(tensor([[ 1, 31, 16,  ...,  0,  0,  0],
         [ 1, 31, 16,  ...,  0,  0,  0],
         [ 1, 31, 16,  ...,  0,  0,  0],
         ...,
         [ 1, 47,  9,  ...,  0,  0,  0],
         [ 1, 47,  9,  ...,  0,  0,  0],
         [ 1, 47,  9,  ...,  0,  0,  0]], dtype=torch.int32),
 tensor([[ 1, 31,  9,  ...,  0,  0,  0],
         [ 1, 31, 20,  ...,  0,  0,  0],
         [ 1,  4, 47,  ...,  0,  0,  0],
         ...,
         [ 1, 47,  9,  ...,  0,  0,  0],
         [ 1,  4, 46,  ...,  0,  0,  0],
         [ 1,  4, 46,  ...,  0,  0,  0]], dtype=torch.int32),
 tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 